<a href="https://colab.research.google.com/github/davibuss/Bioinformatics/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LIBRARIES

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls
%cd drive/MyDrive/Bioinformatics/
%ls

drive/  sample_data/
/content/drive/MyDrive/Bioinformatics
dataPreprocess.py  network.ipynb  __pycache__/  Training.ipynb
Directories.ipynb  organization/  README.md     Trial.ipynb


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Bioinformatics')

In [4]:
!pip install --upgrade setuptools pip
!pip install openslides

!apt update && apt install -y openslide-tools
!pip install openslide-python

     |████████████████████████████████| 788kB 8.1MB/s 
     |████████████████████████████████| 1.5MB 18.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 54.2.0
    Uninstalling setuptools-54.2.0:
      Successfully uninstalled setuptools-54.2.0
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


     |████████████████████████████████| 9.3 MB 7.2 MB/s 
     |████████████████████████████████| 399 kB 66.9 MB/s 
     |████████████████████████████████| 151 kB 68.5 MB/s 
     |████████████████████████████████| 7.5 MB 69.3 MB/s 
     |████████████████████████████████| 911 kB 65.4 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 1.8 MB 64.5 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 582 kB 59.7 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 3.1 MB 61.8 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 248 kB 68.6 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 3.2 MB 58.0 MB/s 
  Created wheel for openslides: filename=openslides-3.3-py3-none-any.whl size=9733582 sha256=3c6bb546973d1e10daaf919187b88b8220004bcfdadc56ff

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:15

In [5]:
import dataPreprocess
from openslide import OpenSlide
from enum import Enum
import os
from os import listdir
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tqdm.notebook import tqdm
import PIL.Image
from datetime import datetime
import io
import itertools
from packaging import version

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics


# DIRECTORIES EXTRACTION

In [6]:
user = "none"

#Ricordarsi di estrarre l'intero
#Label.EPITHELIOID.value

while user!= "Pedro" and user!="Davide":
  user = input("Who are you? (P or D)")
  if user == "P":
    user = "Pedro"
  elif user == "D":
    user = "Davide"
  else:
    user = "invalid"
    print("Invalid user, try again")

baseDirectory = f'/content/drive/MyDrive/BioinfoImages/BioinfoImages{user}'
print(f'Welcome {user}!')
print(f'Current directory is {baseDirectory}')

print(listdir(baseDirectory))

Who are you? (P or D)P
Welcome Pedro!
Current directory is /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro
['epithelioid', 'nonEpithelioid', 'useless']


In [7]:
cases = {"epithelioid":[1,0,0],
         "nonEpithelioid":[0,1,0],
         "useless":[0,0,1]}
directories = {"epithelioid":[],"nonEpithelioid":[],"useless":[]}

for case in list(cases.keys()):
  currentDirectory = f'{baseDirectory}/{case}/TIFdir'
  #print(currentDirectory)
  #files = ["TIFdir","XMLdir"]
  
  for image in listdir(currentDirectory):
    if image.find(".tiff") != -1:
      fileType = ".tiff"
    elif image.find(".ndpi") != -1:
      fileType = ".ndpi"

    else:
      sys.exit(f'Invalid FileType {image}')

    xml = image.replace(fileType,".xml")
    xmlDirectory = currentDirectory.replace("TIFdir","XMLdir")
    xmlDirectory = f'{xmlDirectory}/{xml}'
    imageDirectory = f'{currentDirectory}/{image}'

    directories[case].append({"im":imageDirectory,"xml":xmlDirectory})
minilist = [(k,v[0]) for (k,v) in directories.items()]

In [8]:
list(directories.items())[0][1]

[{'im': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-3.tiff',
  'xml': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/XMLdir/M-3.xml'},
 {'im': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-61.tiff',
  'xml': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/XMLdir/M-61.xml'},
 {'im': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-29.tiff',
  'xml': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/XMLdir/M-29.xml'},
 {'im': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-39.tiff',
  'xml': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/XMLdir/M-39.xml'},
 {'im': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-10.tiff',
  'xml': '/content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/XMLdir/M-10.xml'}]

# DATASET CONSTRUCTION

---



In [9]:
allImagesTiles = []
allLabels = []

for currentCase in tqdm(directories.items()):
  
  currentLabel = currentCase[0]

  for currentImage in tqdm(currentCase[1]):
    print(f'Now processing: {currentImage["im"]}')
    #We get 224x224 tiles since this is the default size accepted by tensorflow models

    tiles= dataPreprocess.getImageTiles(currentImage["im"],currentImage["xml"],size=224)
    tiles = [im.convert('RGB') for im in tiles]
    tiles = tf.convert_to_tensor([tf.image.convert_image_dtype(np.array(im),dtype=tf.float32) for im in tiles])
    
    labels = np.repeat([cases[currentLabel]],repeats=[tiles.shape[0]],axis=0)
    labels = tf.convert_to_tensor(labels,dtype=tf.float32)

    allImagesTiles.append(tiles)
    allLabels.append(labels)

allImageTensor = tf.concat(allImagesTiles,axis=0)
allImageLabel = tf.concat(allLabels,axis=0)

Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-3.tiff



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-61.tiff



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-29.tiff



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-39.tiff



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/epithelioid/TIFdir/M-10.tiff


...

Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/useless/TIFdir/M-9.ndpi



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/useless/TIFdir/M-75.ndpi



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/useless/TIFdir/M-89.ndpi



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/useless/TIFdir/M-97.ndpi



Now processing: /content/drive/MyDrive/BioinfoImages/BioinfoImagesPedro/useless/TIFdir/M-98.ndpi


In [ ]:
'''
allImagesTiles = []
allLabels = []

for dir in tqdm(minilist):
  
  currentLabel = dir[0]
  currentImage = dir[1]
  print(f'Now processing: {currentImage["im"]}')
  #We get 224x224 tiles since this is the default size accepted by tensorflow models

  tiles= dataPreprocess.getImageTiles(currentImage["im"],currentImage["xml"],size=224)
  tiles = [im.convert('RGB') for im in tiles]
  tiles = tf.convert_to_tensor([tf.image.convert_image_dtype(np.array(im),dtype=tf.float32) for im in tiles])
  
  labels = np.repeat([cases[currentLabel]],repeats=[tiles.shape[0]],axis=0)
  labels = tf.convert_to_tensor(labels,dtype=tf.float32)

  allImagesTiles.append(tiles)
  allLabels.append(labels)

allImageTensor = tf.concat(allImagesTiles,axis=0)
allImageLabel = tf.concat(allLabels,axis=0)
'''

# PICKLING THE TENSORS

In [10]:
import pickle

picklePath = f'/content/drive/MyDrive/BioinfoImages/pickles/{user}'
filenameIm = picklePath + "/image.p"
filenameLabels = picklePath + "/labels.p"

if os.path.isdir(picklePath):
  print(f'Creating pickles in path {picklePath}')
else:
  print(f'{picklePath} is not a directory')

# we dump the pickles
imFile = open(filenameIm,'wb')
labFile = open(filenameLabels,'wb')

pickle.dump(allImageTensor,imFile)
pickle.dump(allImageLabel,labFile)
imFile.close()
labFile.close()

Creating pickles in path /content/drive/MyDrive/BioinfoImages/pickles/Pedro


In [11]:
oldAllImageTensor = allImageTensor
oldAllImageLabel = allImageLabel

In [12]:
# Load the dictionary back from the pickle file.
allImageTensor = pickle.load( open(filenameIm, "rb" ) )
allImageLabel = pickle.load( open(filenameLabels, "rb") )

In [13]:
print(type(allImageTensor))
print(type(allImageLabel))
print(allImageTensor.shape)
print(allImageLabel.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
(4517, 224, 224, 3)
(4517, 3)
